### 01. 문자 단위 RNN
- 모든 시점의 입력에 대해서 모든 시점에 대해서 출력을 하는 다대다 RNN 구현
- 대표적으로 품사 태깅, 개체명 인식 등에 사용

#### 1. 문자 단위 RNN
- RNN의 입출력의 단위가 단어 레벨이 아니라 문자 레벨로 하여 RNN을 구현한다면 : 문자 단위 RNN
- RNN 구조 자체가 달라진 것은 아니고 입력과 출력의 단위가 문자로 바뀌었을 뿐!

In [3]:
# 필요한 도구 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

##### 1. 훈련 데이터 전처리하기
- 문자 시퀀스 apple을 입력 받으면 pple!를 출력하는 RNN 구현
- 어떤 의미가 있는 구현은 아니지만 RNN의 동작 이해 목적

In [17]:
# 입력 데이터와 레이블 데이터에 대해 문자 집합(voabulary)을 만든다.
# 여기서 문자 집합은 중복을 제거한 문자들의 집합
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))
print(char_vocab)

문자 집합의 크기 : 5
['!', 'a', 'e', 'l', 'p']


In [8]:
# 하이퍼파라미터 정의
# 이때 입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야 한다.
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [32]:
# 문자 집합에 고유한 정수를 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [23]:
# 나중에 예측 결과를 다시 문자 시퀀스로 보기 위해 반대로 정수로부터 문자를 얻을 수 있는 index_to_char
index_to_char = {}
for key, value in char_to_index.items() :
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [33]:
# 입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [34]:
# 파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받는다. (배치 차원 추가)
# 텐서 연산인 unsqueeze(0)을 통해 해결 가능
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [35]:
# 입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔주기
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)    

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [36]:
# 입력 데이터와 레이블 데이터를 텐서로 바꿔주기
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\ATIV\AppData\Local\Temp\ipykernel_8580\1095725700.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:210.)
  X = torch.FloatTensor(x_one_hot)


In [37]:
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [38]:
# 각 텐서의 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


--------------------------------------------------------------------------------------------------------------

#### 2. 모델 구현하기

In [40]:
class Net(torch.nn.Module) : 
    def __init__(self, input_size, hidden_size, output_size) :
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first = True)
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)
    def forward(self, x) : 
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [41]:
# 클래스로 정의한 모델을 net에 저장
net = Net(input_size, hidden_size, output_size)

In [44]:
# 입력된 모델에 입력을 넣어서 출력 크기 확인
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [45]:
# (1, 5, 5)의 크기를 가지는데 각각 배치 차원, 시점, 출력의 크기
# 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산 : view를 사용하여 배치차원과 시점차원을 하나로 만들기
print(outputs.view(-1, input_size).shape)

torch.Size([5, 5])


In [46]:
# 레이블 데이터의 크기
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [47]:
# 옵티마이저와 손실함수 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [50]:
# 총 100번의 에포크 학습
for i in range(100) :
    optimizer.zero_grad()
    outputs = net(X)
    # view를 하는 이유는 배치 차원 제거를 위해
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    # 기울기 계산
    loss.backward()
    # 파라미터 업데이트
    optimizer.step()
    
    # 모델이 실제 어떻게 예측했는지 확인
    # 최종 예측값인 각 time-step 별 5차원 벡터에 대해 가장 높은 값의 인덱스 선택
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.6941444873809814 prediction:  [[0 0 0 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  !!!!!
1 loss:  1.450325846672058 prediction:  [[3 4 3 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lpl!!
2 loss:  1.2480432987213135 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
3 loss:  1.0311858654022217 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.8284295201301575 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.636077344417572 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.47491854429244995 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.34630411863327026 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.24853095412254333 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.17753739655017853 prediction:  [[4 4 3 2 0]